<a href="https://colab.research.google.com/github/A-wagstaff/CapU-Comp/blob/main/labs/Lab3_neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 3 (NEO)
----------------
#### Name: Alex Wagstaff
#### Date: January 2023

This lab exercise is focused on classes and objects.

**Building on lab 2, and focusing on classes and objects**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithmsply imports all the modules we'll be using...

In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
today = today.strftime('%d-%m-%y')
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-01-19 <class 'datetime.date'>
Today is: 19-01-23


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'cx8zWYh0hvtVWndBCqI5Dfscy7S5Bbv2QLAjqLhP'  # substitute your API key here

today = str(datetime.date.today())  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
# pprint(neos) # Was using these to figure out data structure

9 Near Earth Objects found for 2023-01-19


Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

2 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')
# pprint(hazards) # once again to check data structure

2 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
# id = 2002212 #Static id assigned to use as test case so that data isn't dependent on exercise 1, uncomment line if exercise1 finds 0 hazards
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)
# pprint(data) # Used for manual data check 

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
from typing import ItemsView
from IPython.core.interactiveshell import InteractiveShellABC
data_ex2 = data #Example if all neos returned possess a False value for is_potentially_hazardous_asteroid
close_approach = [item for item in data_ex2['close_approach_data'] if item['orbiting_body'] == 'Earth'] 
close_approach_date = [item['close_approach_date'] for item in close_approach]
miss_distance = [float(item['miss_distance']['kilometers']) for item in close_approach]
miss_dates = sorted(list(zip(miss_distance,close_approach_date)))
print(f'The closest approach of NEO {id} occurs on {miss_dates[0][1]}, where it will come within {round(miss_dates[0][0])}km of Earth')#Not sure if this is required but looks better than a raw tuple
print(miss_dates[0]) #Raw tuple in case that is what you want

The closest approach of NEO 2509352 occurs on 1919-01-04, where it will come within 750820km of Earth
(750819.996016779, '1919-01-04')


## OPTIONAL - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
from datetime import timedelta
delta = timedelta(days = 1)
date1 = datetime.date.today()
date2 = (date1-(7*delta))
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date2}&end_date={date1}&api_key={API_KEY}'
#Copied from earlier in lab for ease
response = requests.request("GET", url, headers={}, data={})
data3 = json.loads(response.text) #data3 because exercise3, wanted it independant so that I could auto complete easier. I know that it isn't best practice but this involved a lot of trial and error.
#Data is now results for past week
n_results = data3['element_count']
print(f'{n_results} Near Earth Objects found for the past week') #Just to make sure that data is updated.
data3_neos = [data3['near_earth_objects'][str(date2+(n*delta))] for n in range(8)]
data3_neos2 = [item for n in range(len(data3_neos)) for item in data3_neos[n]]
data3_cad = [item['close_approach_data'] for item in data3_neos2]
data3_dates = [item['close_approach_date'] for n in range(len(data3_cad)) for item in data3_cad[n] ]
data3_approach = [float(item['miss_distance']['kilometers']) for n in range(len(data3_cad)) for item in data3_cad[n]]
data3_id = [item['id'] for item in data3_neos2]
data3_diam = [float(item['estimated_diameter']['meters']['estimated_diameter_max']) for item in data3_neos2]
data3_min = sorted(list(zip(data3_approach,data3_dates,data3_id,data3_diam)))
print(f'The closest approach for this past week will be by NEO{data3_min[0][2]}. This approach happens on {data3_min[0][1]}, where it will come within approximately {round(data3_min[0][0])}km of Earth.')
print(f'NEO {data3_min[0][2]} is approximately {round(data3_min[0][3],2)}m in diameter.')
# This took me about 10 hours to do. Does this mean that I am doomed? I also managed to pull 5000+ lines of data out of ~2100 lines while testing.

78 Near Earth Objects found for the past week
The closest approach for this past week will be by NEO3673267. This approach happens on 2023-01-14, where it will come within approximately 1819585km of Earth.
NEO 3673267 is approximately 11.75m in diameter.
